# Forest and Natural Resources Management Division Quarterly Accomplishments Script
### This script is broken into 2 major parts: Urban and Community Forestry Stats and FNRM Accomplishments. Part 1 is used to find the total acres of and total number of communities assisted with UC&F. Part 2 is used to create the dot density map showing the quarterly accomplishments. More detailed information can be found in the README on GitHub. https://github.com/jgorman-tfs/FRD-Accomplishments

### Side note for future readers - The division changed its name from FRD to FNRM

## Variables and Imports

In [ ]:
import pandas as pd
import os
#Quarter and fiscal year
qtr = "FY2026Q20"
#Set folder path to new quarter folder
folder_path = rf'D:\ArcGIS_Projects\FRDAccomplishments\{qtr}'
accomp_from_access_path = rf"D:\ArcGIS_Projects\FRDAccomplishments\{qtr}\Accomplishments.XLSX"
ucf_spreadsheet_path = rf"D:\ArcGIS_Projects\FRDAccomplishments\{qtr}\Q1FY26_Urban_OG_TESTING.xlsx"
#This shapefile is provided in sharepoint. If your using your own, you MUST calculate a new field with the County names of each city.
symbology_temp = r"D:\ArcGIS_Projects\FRDAccomplishments\SymbologyTemplate.lyrx"
cities = r"D:\ArcGIS_Projects\FRDAccomplishments\FRDAccomplishments.gdb\Texas_Places_WithCounties"
gdb = r"D:\ArcGIS_Projects\FRDAccomplishments\FRDAccomplishments.gdb"

spam_sheet = "spam_raw"
elmr_sheet = "elmr_raw"

con_ed_activity_list = ["Arbor Day Program", 
                        "UF Presentation", 
                        "Brochure/Web/Newsletter/Media", 
                        "Conference/Workshop/Training", 
                        "Education/Outreach Event or Presentation",
                        "UF Training Given",
                        "Arbor Day/Tree City USA Event"
                       ]
ta_activity_list = ["Tree Planting Event",
                    "Tree Board or Group Activities",
                    "UF Incidental Assist",
                    "UF Individual Assist",
                    "EAB and other Pest Detection/Planning",
                    "Landscape Plan or Site-Specific Issue",
                    "Management Plan",
                    "Tree Inventory or Assessment",
                    "Tree Planting/Maintenance Program",
                    "Tree Ordinance/Policy"
                   ]

## Part 1 : Urban and Community Forestry Stats

### <u><b>Before you begin:<b><u>
### 1. Combine the SPAM and ELMR data into one workbook (xlxs). 
### 2. Make 2 sheets: <u><b> spam_raw and elmr_raw <b><u>
### 3. Copy the city and activity name from the spam table into the spam_raw sheet 
### 4. Copy the office, county, and activity name from the ELMR table to elmr_raw. Change abbreviated offices to the full name (i.e KER -> Kerville

### SPAM SHEET COLUMNS : City , Activity Name
### ELMR SHEET COLUMNS: City, Activity Name, County

### Read the sheets into a single pandas df. Make sure the counts match up.

In [ ]:
dfs = pd.read_excel(
    ucf_spreadsheet_path,
    sheet_name=["spam_raw", "elmr_raw"],
    usecols=["City", "Activity Name"]
)
sheet_counts = {sheet: len(df) for sheet, df in dfs.items()}
    
df = (
    pd.concat(dfs.values(), ignore_index=True)
)

print(df.head())
# Count combined rows
combined_count = len(df)
expected_count = sum(sheet_counts.values())

print(f"\nExpected total rows: {expected_count}")
print(f"Actual combined rows: {combined_count}")

assert combined_count == expected_count, "Row count mismatch!"
print("Row counts match")

### Get the unique activity names

In [ ]:
unique_activities = (
    df[["Activity Name"]]
    .drop_duplicates()
    .sort_values("Activity Name")
    .reset_index(drop=True)
)
print(unique_activities)

### Map the activity type to the activity name. 

### Check to see which activities are not included. You should see General Public/Home/Phone Consultation and Informational Meeting Attended. It was decided that these are not to be included for now, however other categories may appear. Either use best judgement on where to include them or drop them or ask Gretchen/Melissa/Michelle.

In [ ]:
df["Activity Type"] = None

df.loc[
    df["Activity Name"].isin(con_ed_activity_list),
    "Activity Type"
] = "Conservation Education"

df.loc[
    df["Activity Name"].isin(ta_activity_list),
    "Activity Type"
] = "Technical Assistance"

unmapped_counts = (
    df[df["Activity Type"].isna()]["Activity Name"]
    .value_counts()
    .reset_index()
)

unmapped_counts.columns = ["Activity Name", "count"]
print(unmapped_counts)

total_unmapped = unmapped_counts["count"].sum()
print("Total of activities to be dropped: ", total_unmapped)

### Check to make sure the number of rows dropped is expected. This should be the total of unmapped rows minus the total rows of the original data frame.

In [ ]:
df_filtered = df.dropna(subset=["Activity Type"]).reset_index(drop=True)
assert combined_count - total_unmapped == len(df_filtered), "Counts do not line up. The incorrect number of rows were dropped"
print("Counts are as expected")


### Get the Texas_PLaces_WithCounties layer as a dataframe

In [ ]:
fields = ["CityName", "Acres","CountyName"]

# Convert to NumPy array
arr = arcpy.da.TableToNumPyArray(cities, fields)
feature_layer_df = pd.DataFrame(arr)

print(feature_layer_df.head())

### Normalize city names to remove spaces and merge the city list with the Places layer

In [ ]:
df_filtered["City"] = df_filtered["City"].str.replace(" ", "", regex=False)

df_merged = df_filtered.merge(
    feature_layer_df,
    left_on="City",      # column in df_filtered
    right_on="CityName", # column in feature_layer_df
    how="left"
)
df_merged = df_merged.drop(columns=["CityName"])
print(df_merged.head())

### Print out the cities that did not get matched. 

### It is expected to have a few because sometimes SPAM reports the county instead of the city. In that case, you can drop any Texas rows and rows that have counties instead of cities (i.e. Tarrant). There may also be misspellings. In that case you can adjust the spelling on the spreadsheet and run the cells again. 

### MAKE SURE YOU AGREE WITH THE LISTED CITIES TO BE DROPPED

In [ ]:
unmatched_cities = (
    df_merged[df_merged["Acres"].isna()]["City"]
    .value_counts()
    .reset_index()
)

unmatched_cities.columns = ["City", "count"]

print(unmatched_cities)

### Drop the unmatched cities to get a final dataframes; one for the full city list with duplicates (minus unmatched cities and activities) and one of the unique cities. Calculate the total acres assisted for unique cities.

In [ ]:
ucf_df_final = df_merged.dropna(subset=["Acres"]).reset_index(drop=True) 
unique_cities = (
    ucf_df_final[["City","Acres"]]
    .drop_duplicates()
    .sort_values("City")
    .reset_index(drop=True)
)
print(ucf_df_final.head())
print("\nTotal Cities with Duplicates: ", len(ucf_df_final))
print("\nTotal Unique Cities :", len(unique_cities))
total_acres = unique_cities["Acres"].sum()
print("\nTotal Acres Assisted: ", total_acres)

### Send the final dataframes to a single spreadsheet with a sheet for each df

In [ ]:
city_output_filename = f"{qtr}_CityActivities.xlsx"

# Full path to the Excel file
city_output_xlsx = os.path.join(folder_path, city_output_filename)

# Write the Excel file with two sheets
with pd.ExcelWriter(city_output_xlsx, engine="openpyxl") as writer:
    ucf_df_final.to_excel(writer, sheet_name="FinalCityListFull", index=False)
    unique_cities.to_excel(writer, sheet_name="UniqueCities", index=False)

print(f"Excel file saved to: {city_output_xlsx}")

## Part 2: Accomplishments Map

### Read in the SPAM sheet to a df. Normalize the City name column to remove spaces.

In [ ]:
spam_df = pd.read_excel(ucf_spreadsheet_path, sheet_name=spam_sheet)
spam_df["City"] = spam_df["City"].str.replace(" ", "", regex=False)
spam_df.head()

### Read in the SPAM sheet to a df and merge into previously read feature_layer_df (Texas_Places_WithCounties). Check the printed list for names that are cities but didn't get matched. If its a spelling error, fix the source issue and re-read the df back in. If its a city that isn't listed on Texas_Places_WithCounties, add in the case in the next cell and change to name to the correct county. 

In [ ]:
spam_df_merged = spam_df.merge(
    feature_layer_df[["CityName", "CountyName"]],  # only keep relevant columns
    left_on="City",      # column in spam_df
    right_on="CityName", # column in feature_layer_df
    how="left"
)
unmatched_counties = (
    spam_df_merged.loc[spam_df_merged["CountyName"].isna(), "City"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "City", "City": "count"})
)

print("Before fix:")
print(unmatched_counties)
before_fix = spam_df_merged[["City", "CountyName"]].copy()

### Fix the City/County name mix-ups accounting for special cases in the city_to_county dict

In [ ]:
city_to_county = {
    "Texas": "Travis",
    "Cypress": "Harris",
    "Kingwood": "Harris",
    # add more special cases here
}

# Mask for rows with missing CountyName
mask = spam_df_merged["CountyName"].isna()

# Apply mapping or default to city name
spam_df_merged.loc[mask, "CountyName"] = (
    spam_df_merged.loc[mask, "City"].map(city_to_county)  # map special cases
    .fillna(spam_df_merged.loc[mask, "City"])            # default: city name
)

after_fix = spam_df_merged[["City", "CountyName"]]

changes = before_fix.copy()
changes["CountyName_after"] = spam_df_merged["CountyName"]

changes = changes[changes["CountyName"] != changes["CountyName_after"]]

print(changes)
spam_df_merged.head()

### Read in the ELMR sheet to a df

In [ ]:
elmr_df = pd.read_excel(ucf_spreadsheet_path, sheet_name = elmr_sheet)
elmr_df.head()

### Concatenate the ELMR and SPAM df's and get a total count of accomplishments to make sure they match up. You can verify this in the excel sheet if needed.

In [ ]:
elmr_subset = (
    elmr_df[["Activity Name", "County"]]
    .rename(columns={"County": "CountyName"})
)
elmr_count = len(elmr_subset)

spam_subset = spam_df_merged[["Activity Name", "CountyName"]]
spam_count = len(spam_subset)

ucf_accomp_df = pd.concat(
    [elmr_subset, spam_subset],
    ignore_index=True
)
ucf_accomp_total = len(ucf_accomp_df)

print("Number of Accomplishments from ELMR: ", elmr_count)
print("\nNumber of Accomplishments from SPAM: ", spam_count)
print("\nNumber of Unfiltered Accomplishments: ", ucf_accomp_total)
assert ucf_accomp_total == elmr_count + spam_count , "The accomplishment totals do not add up. Check the df concatenation"
ucf_accomp_df.head()

### Map the acitivity names to activity types. Print out the unmapped activities and check to make sure you its okay to drop them. This step should be complete from Part 1 but this is just a double check. 

In [ ]:
ucf_accomp_df["Activity Type"] = None

ucf_accomp_df.loc[
    ucf_accomp_df["Activity Name"].isin(con_ed_activity_list),
    "Activity Type"
] = "Conservation Education"

ucf_accomp_df.loc[
    ucf_accomp_df["Activity Name"].isin(ta_activity_list),
    "Activity Type"
] = "Technical Assistance"

   
ucf_unmapped = ucf_accomp_df[ucf_accomp_df["Activity Type"].isna()]
ucf_unmapped_counts = (
    ucf_unmapped["Activity Name"]
    .value_counts()
    .reset_index()
)

ucf_unmapped_counts.columns = ["Activity Name", "count"]

print(ucf_unmapped_counts)


total_unmapped_ucf_accomp = ucf_unmapped_counts["count"].sum()
print("Total of activities to be dropped:", total_unmapped_ucf_accomp)

mapped_rows = ucf_accomp_df["Activity Type"].notna().sum()
unmapped_rows = ucf_accomp_df["Activity Type"].isna().sum()
print("\nTotal rows:", len(ucf_accomp_df))
print("Mapped rows:", mapped_rows)
print("Unmapped rows:", unmapped_rows)
assert mapped_rows + unmapped_rows == len(ucf_accomp_df), "Mapped rows and Unmapped Rows do not match"
ucf_accomp_df.head()

### Drop the unmapped activities

In [ ]:
ucf_accomp_df_filtered = ucf_accomp_df.dropna(subset=["Activity Type"]).reset_index(drop=True)
print("Total number of filtered Accomplishments: ",len(ucf_accomp_df_filtered))
assert len(ucf_accomp_df_filtered) == len(ucf_accomp_df) - unmapped_rows, "Counts do not line up. The incorrect number of rows were dropped"
print("Counts are as expected")

### Get a count of the Technical Assistance accomplishments

In [ ]:
ta_counts = (
    ucf_accomp_df
    .loc[ucf_accomp_df["Activity Type"] == "Technical Assistance"]
    .groupby("CountyName")
    .size()
    .reset_index(name="Technical Assistance")
)
ta_counts.head()

### Get a count of the Con Ed accomplishments

In [ ]:
con_ed_counts = (
    ucf_accomp_df
    .loc[ucf_accomp_df["Activity Type"] == "Conservation Education"]
    .groupby("CountyName")
    .size()
    .reset_index(name="Conservation Education")
)
con_ed_counts.head()

### Double check to see if there are any mismatches in the County Names of the previous two dfs (ta count and con ed count) and the access DB sheet

In [ ]:
access_df = pd.read_excel(accomp_from_access_path)
access_counties = set(access_df["CountyName"].unique())
comparison_sets = {
    "Conservation Education": set(con_ed_counts["CountyName"].unique()),
    "Technical Assistance": set(ta_counts["CountyName"].unique())
}
# Check for missing counties
missing_counties = {name: counties - access_counties for name, counties in comparison_sets.items()}

# Print results
if all(len(counties) == 0 for counties in missing_counties.values()):
    print("No Missing Counties")
else:
    for name, counties in missing_counties.items():
        if counties:
            print(f"Counties in {name} missing from access_df: {counties}")


### Combine the ucf tables into accomplishments by county

In [ ]:
county_activity_wide = (
    ucf_accomp_df
    .pivot_table(
        index="CountyName",
        columns="Activity Type",
        values="Activity Name",
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)
total_activities_ucf = county_activity_wide.select_dtypes("number").sum().sum()
print("Total activities: ", total_activities_ucf)
county_activity_wide.head(10)

### Read in the accomplishment sheet from the access db

In [ ]:
activity_cols = ["Conservation Education", "Management Plans", "Technical Assistance", "FIA Plots"]
total_activities = access_df[activity_cols].sum().sum()
print("Total number of activities from access db:", total_activities)
print("Number of Rows:", len(access_df))
assert len(access_df) == 254, "Number of rows is not 254. There maybe duplicate entries in the table"
access_df.head()

### Combine all three tables

In [ ]:
access_df = access_df.merge(
    county_activity_wide,
    on="CountyName",
    how="left",
    suffixes=("", "_new")  # keep original columns, new ones get _new
)

# Columns you want to sum
cols_to_sum = ["Conservation Education", "Technical Assistance"]

for col in cols_to_sum:
    access_df[col] = access_df[col].fillna(0) + access_df[f"{col}_new"].fillna(0)
access_df.drop(columns=[f"{col}_new" for col in cols_to_sum], inplace=True)
access_df.head()

### Check to make sure the total counts add up

In [ ]:
final_accomplishment_count_expected = total_activities + total_activities_ucf
print("Total Accomplishments Expected: ", final_accomplishment_count_expected)

final_activity_cols = ["Conservation Education", "Management Plans", "Technical Assistance", "FIA Plots"]
final_accomplishment_count_results = access_df[activity_cols].sum().sum()
print("Total Accomplishments Calculated: ", final_accomplishment_count_results)

assert final_accomplishment_count_expected == final_accomplishment_count_results, "Accomplishment counts do not match. A county join may have failed or you ran the previous cell more than once"
print("\nTotals are in agreement. Proceed to mapping")

### Create a new map and feature layer for the quarter

In [ ]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
counties_new = f"{qtr}_Counties"
arcpy.FeatureClassToFeatureClass_conversion(f"{gdb}\\{"Template"}", gdb, counties_new)
layer_path = os.path.join(gdb, counties_new)
new_map = aprx.createMap(f"{qtr}")
for lyr in new_map.listLayers():
    if lyr.isBasemapLayer:
        new_map.removeLayer(lyr)
new_layer = new_map.addDataFromPath(layer_path)
arcpy.management.ApplySymbologyFromLayer(new_layer, symbology_temp)

### Convert the final table to csv

In [ ]:
final_csv_filename = f"final_accomp_count_{qtr}.csv"
csv_path = os.path.join(folder_path, final_csv_filename)
os.makedirs(folder_path, exist_ok=True)
access_df.to_csv(csv_path, index=False)

### Join the CSV to the new feature layer

In [ ]:
arcpy.management.JoinField(
    in_data=layer_path,
    in_field="CountyName",
    join_table=csv_path,
    join_field="CountyName",
    fields="'Conservation Education';'Management Plans';'Technical Assistance';'FIA Plots'",
    fm_option="NOT_USE_FM",
    field_mapping=None,
    index_join_fields="NO_INDEXES"
)

### Read the feature layer into a df for a final count check.

In [ ]:
fields_list = ["Conservation_Education", "Management_Plans","Technical_Assistance","FIA_Plots"]
final_arr = arcpy.da.TableToNumPyArray(layer_path, fields_list)
final_layer_df = pd.DataFrame(final_arr)
final_check = final_layer_df.sum().sum()
print("Total Accomplishments Final Check: ", final_check)
assert final_check == final_accomplishment_count_results, "Final results do not agree. Make sure the number of accomplishments in the feature class matches Total Accomplishments Calculated"
print("Final Check Passed. Good to Continue")
final_layer_df.head()

### Create the layout and update the contents

In [ ]:
# Extract year and quarter number
fy = qtr[:2]            # "FY"
year = qtr[2:6]         # "2026"
quarter = qtr[-1]       # "1"

# Combine into new format
formatted_qtr = f"{fy} {year} - Quarter {quarter}"

print(formatted_qtr)
# Output: FY 2026 - Quarter 1
layout = aprx.listLayouts("LayoutTemplate")[0]

map_frame = layout.listElements("MAPFRAME_ELEMENT")[0]
map_frame.map = new_map
title_elem = next((t for t in layout.listElements("TEXT_ELEMENT") if t.name == "Text 1"), None)
if title_elem:
    title_elem.text = formatted_qtr

### Export the map to PDF

In [ ]:
output_pdf_path = os.path.join(folder_path, f"Quarterly_Accomplishments_{qtr}.pdf")
layout.exportToPDF(output_pdf_path, resolution=300)

print(f"Layout updated and exported to {output_pdf_path}")

## DONE